## How to perform File Transfers during workflows

We can perform file transfer operations pre or post electron execution here we illustrate how to perform file transfer using Rsync locally and remotely via SSH. 

### Local File Transfers (Rsync)

We first define a source & destination filepath where we want to transfer a file from the `source_filepath` location to the `destination_filepath` location as well as create an empty file in `source_filepath` to have a file to transfer.

In [ ]:
from pathlib import Path

# define source & destination filepaths 
source_filepath = Path('./my_source_file').resolve()
destination_filepath = Path('./my_dest_file').resolve()

# create an example file
source_filepath.touch()

We then run a workflow by defining our `FileTransfer` operation which defaults to the local `Rsync` strategy.

In [ ]:
import covalent as ct 

In [ ]:
@ct.electron(
        files=[ct.fs.FileTransfer(str(source_filepath), str(destination_filepath))]
)
def my_file_transfer_task():
    pass
      
@ct.lattice()
def my_workflow():
    return my_file_transfer_task()

ct.dispatch(my_workflow)()

After executing the workflow we now see a copy of the file (`source_filepath`) located in `my_dest_file`. This file transfer occured prior to electon execution.

### Remote File Transfers (Rsync SSH)

Similarly we can perform file transfers using Rsync via SSH in order to transfer a file located in `source_filepath` to a remote host's filesystem located at `/home/ubuntu/my_dest_file`

```python
strategy = ct.fs_strategies.Rsync(user='ubuntu', host='44.202.86.215', private_key_path='/path/to/private/key')

@ct.electron(
        files=[ct.fs.TransferToRemote('/home/ubuntu/my_dest_file', str(source_filepath), strategy=strategy)]
)
def my_remote_file_transfer_task():
    pass
      
@ct.lattice()
def my_workflow():
    return my_remote_file_transfer_task()

ct.dispatch(my_workflow)()
```

After workflow execution the file located at `source_filepath` will be transfered to host `44.202.86.215` in the host's filesystem (`/home/ubuntu/my_dest_file`). This file transfer occurs after electron execution.